In [1]:
import pandas as pd
import os
import pandas as pd
import numpy as np
from typing import Dict
import pandas as pd
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sqlalchemy import create_engine
import kagglehub


c:\Users\PC\Datawarehouse\Transform\ETL_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [129]:


# 1. Định nghĩa Tên Dataset
# Thay thế bằng định danh chính xác của bộ dữ liệu bạn đã xem trên Kaggle
DATASET_REF = "nosbielcs/brazilian-delivery-center" 

# 2. Gọi API để Tải và Giải nén Dataset
# Hàm này sẽ tải file zip, tự động giải nén và trả về đường dẫn tới thư mục
print(f"Bắt đầu tải dataset: {DATASET_REF}")
try:
    # 'path' là đường dẫn cục bộ tới thư mục chứa các file .csv đã giải nén
    download_path = kagglehub.dataset_download(DATASET_REF)
    
    # Cập nhật folder_path tới nơi KaggleHub đã tải dataset
    # Ta sử dụng đường dẫn này thay cho đường dẫn 'C:\Users\PC\Datawarehouse\Raw'
    folder_path = download_path

    print(f"Tải và giải nén thành công. Đường dẫn file: {folder_path}")

except Exception as e:
    print(f"LỖI khi tải dữ liệu từ Kaggle API: {e}")
    # Nếu API lỗi, bạn có thể cân nhắc fallback về đường dẫn thủ công nếu cần
    folder_path = r"C:\Users\PC\Datawarehouse\Raw"
    print(f"Sử dụng đường dẫn cục bộ dự phòng: {folder_path}")


Bắt đầu tải dataset: nosbielcs/brazilian-delivery-center


 33%|███▎      | 9.00M/27.7M [00:03<00:08, 2.39MB/s]


KeyboardInterrupt: 

In [2]:
folder_path = r"C:\Users\PC\Datawarehouse\Raw"

In [3]:
# Danh sách tên các tệp (chúng ta sẽ giả định tệp có đuôi .csv)
file_names = ["channels", "deliveries", "drivers", "hubs", "orders", "payments", "stores"]
dataframes = {}

print("Bắt đầu đọc các tệp CSV và xử lý lỗi Encoding...")

for file_name in file_names:
    # Thay đổi đuôi tệp từ .xlsx sang .csv
    full_path = os.path.join(folder_path, f"{file_name}.csv")
    
    # 💡 THAM SỐ QUAN TRỌNG: encoding
    encoding_to_use = 'utf-8'
    
    # Áp dụng encoding 'latin-1' (hoặc 'iso-8859-1') cho các tệp bị lỗi (hubs, stores)
    if file_name in ["hubs", "stores"]:
        encoding_to_use = 'latin-1'
    
    try:
        # Sử dụng pd.read_csv() và áp dụng encoding đã chọn
        df = pd.read_csv(full_path, encoding=encoding_to_use)
        dataframes[file_name] = df
        print(f"✅ Đã đọc thành công tệp: **{file_name}.csv** ({len(df)} hàng) với encoding **{encoding_to_use}**")
        
    except FileNotFoundError:
        print(f"❌ Lỗi: Không tìm thấy tệp tại đường dẫn: {full_path}")
    except Exception as e:
        # Nếu Latin-1 vẫn không được, thử Windows-1252
        if encoding_to_use == 'latin-1':
            try:
                df = pd.read_csv(full_path, encoding='windows-1252')
                dataframes[file_name] = df
                print(f"✅ Đã đọc thành công tệp: **{file_name}.csv** ({len(df)} hàng) với encoding **windows-1252**")
            except Exception as e_w:
                print(f"❌ Lỗi đọc tệp **{file_name}** với Windows-1252: {e_w}")
        else:
            print(f"❌ Lỗi đọc tệp **{file_name}** với {encoding_to_use}: {e}")

print("\nHoàn thành quá trình import.")

Bắt đầu đọc các tệp CSV và xử lý lỗi Encoding...
✅ Đã đọc thành công tệp: **channels.csv** (40 hàng) với encoding **utf-8**
✅ Đã đọc thành công tệp: **deliveries.csv** (378843 hàng) với encoding **utf-8**
✅ Đã đọc thành công tệp: **drivers.csv** (4824 hàng) với encoding **utf-8**
✅ Đã đọc thành công tệp: **hubs.csv** (32 hàng) với encoding **latin-1**
✅ Đã đọc thành công tệp: **orders.csv** (368999 hàng) với encoding **utf-8**
✅ Đã đọc thành công tệp: **payments.csv** (400834 hàng) với encoding **utf-8**
✅ Đã đọc thành công tệp: **stores.csv** (951 hàng) với encoding **latin-1**

Hoàn thành quá trình import.


In [4]:
data_orders = dataframes['orders']
data_payments = dataframes['payments']
data_channels = dataframes['channels']
data_deliveries = dataframes['deliveries']
data_drivers = dataframes['drivers']
data_hubs = dataframes['hubs']
data_stores = dataframes['stores']


In [43]:
def remove_duplicates(dataframes_dict):
    """Kiểm tra và loại bỏ các hàng trùng lặp trong tất cả các DataFrame."""
    print("\n--- Bắt đầu kiểm tra và loại bỏ bản ghi trùng lặp ---")
    
    cleaned_dataframes = {}

    for name, df in dataframes_dict.items():
        initial_rows = len(df)
        duplicate_count = df.duplicated().sum()

        if duplicate_count > 0:
            df_cleaned = df.drop_duplicates(keep='first', inplace=False)
            deleted_rows = initial_rows - len(df_cleaned)
            
            print(f"Bảng **{name}**: Đã tìm thấy và xóa **{deleted_rows}** bản ghi trùng lặp.")
            cleaned_dataframes[name] = df_cleaned
        else:
            print(f"Bảng **{name}**: Không có bản ghi trùng lặp. ({initial_rows} hàng)")
            cleaned_dataframes[name] = df
            
    print("--- Hoàn thành xử lý trùng lặp ---")
    return cleaned_dataframes

In [44]:
# Giả sử đây là dictionary chứa tất cả các DataFrame của bạn sau khi import
# dataframes = {'channels': df_channels, 'deliveries': df_deliveries, 'hubs': df_hubs, ...}

# THỰC THI HÀM: Áp dụng việc xóa trùng lặp cho TẤT CẢ các bảng
print("\n🔥 BẮT ĐẦU GIAI ĐOẠN TRANSFORM: Xử lý Trùng lặp")
dataframes = remove_duplicates(dataframes)

print("\nKiểm tra nhanh kết quả:")
print(f"Số lượng bảng đã được xử lý: {len(dataframes)}")


🔥 BẮT ĐẦU GIAI ĐOẠN TRANSFORM: Xử lý Trùng lặp

--- Bắt đầu kiểm tra và loại bỏ bản ghi trùng lặp ---
Bảng **channels**: Không có bản ghi trùng lặp. (40 hàng)
Bảng **deliveries**: Không có bản ghi trùng lặp. (378843 hàng)
Bảng **drivers**: Không có bản ghi trùng lặp. (4824 hàng)
Bảng **hubs**: Không có bản ghi trùng lặp. (32 hàng)
Bảng **orders**: Không có bản ghi trùng lặp. (368999 hàng)
Bảng **payments**: Không có bản ghi trùng lặp. (400834 hàng)
Bảng **stores**: Không có bản ghi trùng lặp. (951 hàng)
--- Hoàn thành xử lý trùng lặp ---

Kiểm tra nhanh kết quả:
Số lượng bảng đã được xử lý: 7


## Xử lý bảng Orders

In [5]:
data_orders.head(10)

,order_id,store_id,channel_id,payment_order_id,delivery_order_id,order_status,order_amount,order_delivery_fee,order_delivery_cost,order_created_hour,...,order_moment_delivering,order_moment_delivered,order_moment_finished,order_metric_collected_time,order_metric_paused_time,order_metric_production_time,order_metric_walking_time,order_metric_expediton_speed_time,order_metric_transit_time,order_metric_cycle_time
0,68405119,3512,5,68405119,68405119,CANCELED,62.7,0.00,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,68405123,3512,5,68405123,68405123,CANCELED,62.7,0.00,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,68405206,3512,5,68405206,68405206,CANCELED,115.5,0.00,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,68405465,3401,5,68405465,68405465,CANCELED,55.9,0.00,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,68406064,3401,5,68406064,68406064,CANCELED,37.9,0.00,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,68408108,786,5,68408108,68408108,CANCELED,80.0,9.90,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,68408109,1125,5,68408109,68408109,CANCELED,71.0,9.90,NaN,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,68409030,1064,35,68409030,68409030,CANCELED,214.8,0.01,0.0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,68410055,2181,35,68410055,68410055,FINISHED,394.8,0.01,6.0,2,...,1/2/2021 6:35:49 PM,NaN,1/2/2021 6:57:34 PM,6.63,4.55,2391.25,7.17,11.72,21.75,2424.72
9,68412121,1152,5,68412121,68412121,CANCELED,25.5,9.90,NaN,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368999 entries, 0 to 368998
Data columns (total 29 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   order_id                           368999 non-null  int64  
 1   store_id                           368999 non-null  int64  
 2   channel_id                         368999 non-null  int64  
 3   payment_order_id                   368999 non-null  int64  
 4   delivery_order_id                  368999 non-null  int64  
 5   order_status                       368999 non-null  object 
 6   order_amount                       368999 non-null  float64
 7   order_delivery_fee                 368999 non-null  float64
 8   order_delivery_cost                361794 non-null  float64
 9   order_created_hour                 368999 non-null  int64  
 10  order_created_minute               368999 non-null  int64  
 11  order_created_day                  3689

In [7]:
data_orders.isnull().sum().sort_values(ascending=False)

order_moment_delivered               349398
order_metric_walking_time             74056
order_metric_paused_time              71405
order_moment_in_expedition            67429
order_metric_collected_time           51492
order_moment_collected                42894
order_metric_expediton_speed_time     34582
order_metric_transit_time             25857
order_moment_delivering               25316
order_metric_production_time          25107
order_moment_ready                    25106
order_metric_cycle_time               15619
order_moment_finished                 15599
order_moment_accepted                  9461
order_delivery_cost                    7205
order_created_minute                      0
order_created_hour                        0
order_created_month                       0
order_created_day                         0
order_status                              0
delivery_order_id                         0
order_id                                  0
payment_order_id                

In [48]:
missing_data = data_orders.isnull().sum()
percent_missing = (missing_data / len(data_orders)) * 100
percent_missing = percent_missing[percent_missing > 0].sort_values(ascending=False)
print("---- Tỉ lệ dữ liệu thiếu trong bảng orders ----")
percent_missing

---- Tỉ lệ dữ liệu thiếu trong bảng orders ----


order_moment_delivered               94.688061
order_metric_walking_time            20.069431
order_metric_paused_time             19.351001
order_moment_in_expedition           18.273491
order_metric_collected_time          13.954509
order_moment_collected               11.624422
order_metric_expediton_speed_time     9.371841
order_metric_transit_time             7.007336
order_moment_delivering               6.860723
order_metric_production_time          6.804083
order_moment_ready                    6.803812
order_metric_cycle_time               4.232803
order_moment_finished                 4.227383
order_moment_accepted                 2.563964
order_delivery_cost                   1.952580
dtype: float64

- Xóa cột không cần thiết
  - Xóa cột `order_momment_dilivered` do cột có tới 90% dữ liệu thiếu > không có ý nghĩa trong kinh doanh

In [8]:
data_orders.drop(columns=['order_moment_delivered','delivery_order_id','payment_order_id'], inplace=True)

* Định dạng lại dữ liệu

In [9]:

data_orders['order_moment_created'] = pd.to_datetime(data_orders['order_moment_created'],format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
data_orders['order_moment_accepted'] = pd.to_datetime(data_orders['order_moment_accepted'],format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
data_orders['order_moment_finished'] = pd.to_datetime(data_orders['order_moment_finished'],format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
data_orders['order_moment_ready'] = pd.to_datetime(data_orders['order_moment_ready'],format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
data_orders['order_moment_collected'] = pd.to_datetime(data_orders['order_moment_collected'],format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
data_orders['order_moment_delivering'] = pd.to_datetime(data_orders['order_moment_delivering'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')
data_orders['order_moment_in_expedition'] = pd.to_datetime(data_orders['order_moment_in_expedition'], format='%m/%d/%Y %I:%M:%S %p', errors='coerce')




In [10]:
data_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368999 entries, 0 to 368998
Data columns (total 26 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   order_id                           368999 non-null  int64         
 1   store_id                           368999 non-null  int64         
 2   channel_id                         368999 non-null  int64         
 3   order_status                       368999 non-null  object        
 4   order_amount                       368999 non-null  float64       
 5   order_delivery_fee                 368999 non-null  float64       
 6   order_delivery_cost                361794 non-null  float64       
 7   order_created_hour                 368999 non-null  int64         
 8   order_created_minute               368999 non-null  int64         
 9   order_created_day                  368999 non-null  int64         
 10  order_created_month 

* Thêm một cột chỉ dẫn xem đơn hàng có được hoàn thành không 

In [11]:
data_orders['is_finished'] = data_orders['order_status'].apply(lambda x: 1 if x == 'FINISHED' else 0)

In [12]:
data_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368999 entries, 0 to 368998
Data columns (total 27 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   order_id                           368999 non-null  int64         
 1   store_id                           368999 non-null  int64         
 2   channel_id                         368999 non-null  int64         
 3   order_status                       368999 non-null  object        
 4   order_amount                       368999 non-null  float64       
 5   order_delivery_fee                 368999 non-null  float64       
 6   order_delivery_cost                361794 non-null  float64       
 7   order_created_hour                 368999 non-null  int64         
 8   order_created_minute               368999 non-null  int64         
 9   order_created_day                  368999 non-null  int64         
 10  order_created_month 

* Thực hiện kiểm tra phân tích đối với đơn hàng đã hoàn thành

In [13]:
data_orders_finished = data_orders[data_orders['is_finished'] == 1]

In [14]:
data_orders[['order_id','store_id','channel_id','order_status','order_amount']].head(10)

,order_id,store_id,channel_id,order_status,order_amount
0,68405119,3512,5,CANCELED,62.7
1,68405123,3512,5,CANCELED,62.7
2,68405206,3512,5,CANCELED,115.5
3,68405465,3401,5,CANCELED,55.9
4,68406064,3401,5,CANCELED,37.9
5,68408108,786,5,CANCELED,80.0
6,68408109,1125,5,CANCELED,71.0
7,68409030,1064,35,CANCELED,214.8
8,68410055,2181,35,FINISHED,394.8
9,68412121,1152,5,CANCELED,25.5


* Kiểm tra NULL

In [15]:
data_orders_finished.isnull().sum()

order_id                                 0
store_id                                 0
channel_id                               0
order_status                             0
order_amount                             0
order_delivery_fee                       0
order_delivery_cost                   6000
order_created_hour                       0
order_created_minute                     0
order_created_day                        0
order_created_month                      0
order_created_year                       0
order_moment_created                     0
order_moment_accepted                 1368
order_moment_ready                   11043
order_moment_collected               27764
order_moment_in_expedition           52121
order_moment_delivering               9654
order_moment_finished                    3
order_metric_collected_time          36232
order_metric_paused_time             55531
order_metric_production_time         11044
order_metric_walking_time            58671
order_metri

In [16]:

missing_values_count = data_orders_finished.isnull().sum()
total_rows = len(data_orders_finished)
missing_percentage = (missing_values_count / total_rows) * 100
print("--- Tỷ lệ Dữ liệu Mất trong bảng 'orders ( finished )' ---")
print(missing_percentage.sort_values(ascending=False))

--- Tỷ lệ Dữ liệu Mất trong bảng 'orders ( finished )' ---
order_metric_walking_time            16.666951
order_metric_paused_time             15.774956
order_moment_in_expedition           14.806261
order_metric_collected_time          10.292597
order_moment_collected                7.887052
order_metric_expediton_speed_time     5.354241
order_metric_production_time          3.137322
order_moment_ready                    3.137038
order_metric_transit_time             2.794728
order_moment_delivering               2.742458
order_delivery_cost                   1.704449
order_moment_accepted                 0.388614
order_metric_cycle_time               0.103119
order_moment_finished                 0.000852
order_created_day                     0.000000
order_created_month                   0.000000
order_amount                          0.000000
order_status                          0.000000
channel_id                            0.000000
order_id                              0.000000
s

- Đối với các trường thời gian thì sẽ không xử lý NULL mà giữ nguyên nó
- Đối với trường measure như `order_delivery_cost`, do tỉ lệ thiếu ít, nên ta sẽ điền giá trị trung vị 

In [17]:
fix_null = data_orders_finished['order_delivery_cost'].median()
data_orders_finished_notfix = data_orders_finished.copy()
data_orders_finished['order_delivery_cost'].fillna(fix_null, inplace=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_25700\1923428877.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_orders_finished['order_delivery_cost'].fillna(fix_null, inplace=True)
C:\Users\PC\AppData\Local\Temp\ipykernel_25700\1923428877.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_orders_finished['order_delivery_cost'].fillna

In [18]:
data_orders_finished_notfix[['order_id','store_id','channel_id','order_status','order_amount','order_delivery_cost']][data_orders_finished_notfix['order_delivery_cost'].isnull()].head(10)

,order_id,store_id,channel_id,order_status,order_amount,order_delivery_cost
669,68493525,149,5,FINISHED,45.90,NaN
682,68495519,631,5,FINISHED,203.70,NaN
695,68498710,3265,5,FINISHED,24.90,NaN
701,68500865,495,5,FINISHED,126.30,NaN
767,68511483,3091,5,FINISHED,29.90,NaN
798,68516950,274,5,FINISHED,70.80,NaN
960,68538068,493,15,FINISHED,75.80,NaN
976,68539343,493,15,FINISHED,123.75,NaN
1150,68555142,1324,5,FINISHED,42.00,NaN
1540,68623893,1037,5,FINISHED,175.85,NaN


In [19]:
data_orders_finished[['order_id','store_id','channel_id','order_status','order_amount','order_delivery_cost']][data_orders_finished_notfix['order_delivery_cost'].isnull()].head(10)

,order_id,store_id,channel_id,order_status,order_amount,order_delivery_cost
669,68493525,149,5,FINISHED,45.90,7.39
682,68495519,631,5,FINISHED,203.70,7.39
695,68498710,3265,5,FINISHED,24.90,7.39
701,68500865,495,5,FINISHED,126.30,7.39
767,68511483,3091,5,FINISHED,29.90,7.39
798,68516950,274,5,FINISHED,70.80,7.39
960,68538068,493,15,FINISHED,75.80,7.39
976,68539343,493,15,FINISHED,123.75,7.39
1150,68555142,1324,5,FINISHED,42.00,7.39
1540,68623893,1037,5,FINISHED,175.85,7.39


In [20]:
data_orders_finished.isnull().sum() 

order_id                                 0
store_id                                 0
channel_id                               0
order_status                             0
order_amount                             0
order_delivery_fee                       0
order_delivery_cost                      0
order_created_hour                       0
order_created_minute                     0
order_created_day                        0
order_created_month                      0
order_created_year                       0
order_moment_created                     0
order_moment_accepted                 1368
order_moment_ready                   11043
order_moment_collected               27764
order_moment_in_expedition           52121
order_moment_delivering               9654
order_moment_finished                    3
order_metric_collected_time          36232
order_metric_paused_time             55531
order_metric_production_time         11044
order_metric_walking_time            58671
order_metri

* Thêm cột `Cycle_Time_Category` dựa vào thời gian giao hàng

In [21]:
# Kiểm tra các điểm phân vị để xác định ngưỡng
print(data_orders_finished['order_metric_cycle_time'].describe(percentiles=[.25, .5, .75, .90]))

count    351657.000000
mean        155.843596
std        1225.229132
min           0.370000
25%          32.870000
50%          42.180000
75%          55.550000
90%          83.170000
max      154277.320000
Name: order_metric_cycle_time, dtype: float64


In [22]:


# 1. Khai báo các ngưỡng thực tế từ kết quả
P25 = 32.87
P75 = 55.55

# 2. Định nghĩa các điều kiện
conditions = [
    # 25% nhanh nhất
    (data_orders_finished['order_metric_cycle_time'] <= 32.87),
    # 50% ở giữa
    (data_orders_finished['order_metric_cycle_time'] > 32.87) & (data_orders_finished['order_metric_cycle_time'] <= 55.55),
    # Xử lý giá trị thiếu (NaN)
    (data_orders_finished['order_metric_cycle_time'].isnull())
]

# 3. Định nghĩa các kết quả
choices = ['Fast', 'Medium', 'Missing Data']

# 4. Áp dụng phân loại
# Giá trị default sẽ là 'Slow' (tất cả các giá trị lớn hơn P75 và không phải NaN)
data_orders_finished['Cycle_Time_Category'] = np.select(conditions, choices, default='Slow')

# In kết quả kiểm tra
print(data_orders_finished['Cycle_Time_Category'].value_counts())

Cycle_Time_Category
Medium          175770
Fast             88024
Slow             87863
Missing Data       363
Name: count, dtype: int64


C:\Users\PC\AppData\Local\Temp\ipykernel_25700\1275661988.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_orders_finished['Cycle_Time_Category'] = np.select(conditions, choices, default='Slow')


In [23]:
data_orders_finished.head(20)

,order_id,store_id,channel_id,order_status,order_amount,order_delivery_fee,order_delivery_cost,order_created_hour,order_created_minute,order_created_day,...,order_moment_finished,order_metric_collected_time,order_metric_paused_time,order_metric_production_time,order_metric_walking_time,order_metric_expediton_speed_time,order_metric_transit_time,order_metric_cycle_time,is_finished,Cycle_Time_Category
8,68410055,2181,35,FINISHED,394.80,0.01,6.00,2,32,1,...,2021-01-02 18:57:34,6.63,4.55,2391.25,7.17,11.72,21.75,2424.72,1,Slow
17,68412721,631,5,FINISHED,195.05,11.90,10.93,14,12,1,...,2021-01-01 16:12:36,0.27,10.22,26.07,0.83,11.05,83.30,120.42,1,Slow
18,68413340,631,5,FINISHED,46.90,11.90,11.36,14,14,1,...,2021-01-01 15:31:54,7.03,3.10,14.62,9.57,12.67,49.78,77.05,1,Slow
19,68414018,3265,5,FINISHED,45.80,0.00,10.28,14,17,1,...,2021-01-01 15:09:20,10.22,3.03,27.48,10.25,13.28,11.05,51.82,1,Medium
20,68414309,236,5,FINISHED,94.90,11.90,6.00,14,21,1,...,2021-01-01 14:50:20,2.37,1.45,15.78,5.33,6.78,6.73,29.30,1,Fast
21,68414512,631,5,FINISHED,45.90,11.90,11.63,14,24,1,...,2021-01-01 15:49:47,0.08,39.75,22.10,0.10,39.85,23.82,85.77,1,Slow
22,68414563,955,5,FINISHED,26.90,0.00,6.00,14,24,1,...,2021-01-01 15:30:56,1.23,NaN,39.03,NaN,5.92,21.55,66.50,1,Slow
24,68415103,631,5,FINISHED,103.60,11.90,6.33,14,31,1,...,2021-01-01 15:38:55,0.42,3.93,24.82,0.45,4.38,38.17,67.37,1,Slow
25,68415140,631,5,FINISHED,187.40,11.90,3.72,14,31,1,...,2021-01-01 16:06:40,1.57,4.18,30.93,2.52,6.70,57.17,94.82,1,Slow
26,68415344,631,5,FINISHED,83.40,11.90,11.32,14,33,1,...,2021-01-01 16:18:52,0.52,28.83,22.88,0.53,29.38,53.08,105.35,1,Slow


* Tạo thêm trường `day_of_week_created`

In [24]:
#Tạo cột ngày trong tuần từ cột order_moment_created
data_orders['day_of_week_created'] = data_orders['order_moment_created'].dt.day_name()

* Tạo thêm `time_band` xác định xem khách hàng mua hàng vào thời điểm nào trong ngày

In [25]:


def get_time_band(hour):
    """
    Phân loại giờ đặt hàng thành Morning, Afternoon, hoặc Night.
    """
    if 6 <= hour < 12:
        return 'Morning'    # 6:00 đến 11:59
    elif 12 <= hour < 18:
        return 'Afternoon'  # 12:00 đến 17:59
    else:
        return 'Night'      # 18:00 đến 5:59
        
# Áp dụng hàm lên cột 'order_created_hour' để tạo cột mới 'time_band_created'
data_orders['time_band_created'] = data_orders['order_created_hour'].apply(get_time_band)

# Hiển thị kết quả (chỉ hiện các cột liên quan để kiểm tra)
print(data_orders[['order_created_hour', 'time_band_created']].head(10))

   order_created_hour time_band_created
0                   0             Night
1                   0             Night
2                   0             Night
3                   0             Night
4                   0             Night
5                   0             Night
6                   0             Night
7                   1             Night
8                   2             Night
9                  13         Afternoon


In [26]:
data_orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368999 entries, 0 to 368998
Data columns (total 29 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   order_id                           368999 non-null  int64         
 1   store_id                           368999 non-null  int64         
 2   channel_id                         368999 non-null  int64         
 3   order_status                       368999 non-null  object        
 4   order_amount                       368999 non-null  float64       
 5   order_delivery_fee                 368999 non-null  float64       
 6   order_delivery_cost                361794 non-null  float64       
 7   order_created_hour                 368999 non-null  int64         
 8   order_created_minute               368999 non-null  int64         
 9   order_created_day                  368999 non-null  int64         
 10  order_created_month 

* Cập nhật bảng `orders` gốc

In [27]:

# Giả sử DataFrame gốc của bạn là data_orders
P25 = 32.87 # (Ví dụ)
P75 = 55.55 # (Ví dụ)

# 1. TẠO CỘT MỚI VÀ TÍNH TOÁN TRỰC TIẾP TRÊN DF GỐC
# Tạo điều kiện chỉ tính toán cho các đơn hàng đã HOÀN THÀNH
conditions = [
    (data_orders['is_finished'] == 1) & (data_orders['order_metric_cycle_time'] <= P25),
    (data_orders['is_finished'] == 1) & (data_orders['order_metric_cycle_time'] > P25) & (data_orders['order_metric_cycle_time'] <= P75),
    (data_orders['is_finished'] == 1) & (data_orders['order_metric_cycle_time'] > P75)
]
choices = ['Fast', 'Medium', 'Slow']

# Gán giá trị: Default='Chậm' (cho đơn Finished > P75) hoặc NaN (cho đơn chưa Finished)
# Đặt giá trị mặc định ban đầu là 'Missing Data' hoặc 'NaN'
data_orders['Cycle_Time_Category'] = np.select(conditions, choices, default='Slow')

# 2. XỬ LÝ CÁC ĐƠN HÀNG CHƯA HOÀN THÀNH (is_finished == 0)
# Xác định điều kiện cho các đơn hàng chưa hoàn thành/bị hủy
unfinished_condition = data_orders['is_finished'] == 0

# Gán giá trị 'Not Applicable' cho các đơn chưa hoàn thành
data_orders.loc[unfinished_condition, 'Cycle_Time_Category'] = 'Not Applicable'

# 3. XỬ LÝ DỮ LIỆU THIẾU CÒN LẠI (Nếu Cycle Time là NaN trong đơn Finished)
# Điều kiện: Đơn đã finished VÀ Cycle_Time_Category vẫn là NaN (do Cycle Time gốc là NaN)
missing_condition = (data_orders['is_finished'] == 1) & (data_orders['Cycle_Time_Category'].isnull())

# Gán giá trị 'Missing Data' cho các đơn Finished bị thiếu Cycle Time
data_orders.loc[missing_condition, 'Cycle_Time_Category'] = 'Missing Data'

In [28]:
data_orders['order_delivery_cost'].fillna(fix_null, inplace=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_25700\2863223968.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_orders['order_delivery_cost'].fillna(fix_null, inplace=True)


In [70]:
data_orders[['order_id','store_id','channel_id','order_status']].head(20)    

,order_id,store_id,channel_id,order_status
0,68405119,3512,5,CANCELED
1,68405123,3512,5,CANCELED
2,68405206,3512,5,CANCELED
3,68405465,3401,5,CANCELED
4,68406064,3401,5,CANCELED
5,68408108,786,5,CANCELED
6,68408109,1125,5,CANCELED
7,68409030,1064,35,CANCELED
8,68410055,2181,35,FINISHED
9,68412121,1152,5,CANCELED


In [29]:
dataframes['orders'] = data_orders

In [72]:
dataframes['orders'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368999 entries, 0 to 368998
Data columns (total 30 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   order_id                           368999 non-null  int64         
 1   store_id                           368999 non-null  int64         
 2   channel_id                         368999 non-null  int64         
 3   order_status                       368999 non-null  object        
 4   order_amount                       368999 non-null  float64       
 5   order_delivery_fee                 368999 non-null  float64       
 6   order_delivery_cost                368999 non-null  float64       
 7   order_created_hour                 368999 non-null  int64         
 8   order_created_minute               368999 non-null  int64         
 9   order_created_day                  368999 non-null  int64         
 10  order_created_month 

## Xử lý bảng "Channel"

In [73]:
data_channels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   channel_id    40 non-null     int64 
 1   channel_name  40 non-null     object
 2   channel_type  40 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.1+ KB


In [74]:
data_channels.head(20)

,channel_id,channel_name,channel_type
0,1,OTHER PLACE,OWN CHANNEL
1,2,PHONE PLACE,OWN CHANNEL
2,3,WHATS PLACE,OWN CHANNEL
3,4,FACE PLACE,OWN CHANNEL
4,5,FOOD PLACE,MARKETPLACE
5,6,STORE PLACE,OWN CHANNEL
6,7,BERLIN PLACE,OWN CHANNEL
7,8,MADRID PLACE,OWN CHANNEL
8,9,THINK PLACE,OWN CHANNEL
9,10,LISBON PLACE,OWN CHANNEL


- Kiểm tra NULL

In [75]:
data_channels.isnull().sum()

channel_id      0
channel_name    0
channel_type    0
dtype: int64

## Xử lý bảng "Deliveries"


In [76]:
data_deliveries.head(10)

,delivery_id,delivery_order_id,driver_id,delivery_distance_meters,delivery_status
0,2174658,68413340,8378.0,5199.0,DELIVERED
1,2174660,68414309,2473.0,410.0,DELIVERED
2,2174661,68416230,7615.0,3784.0,DELIVERED
3,2174663,68412721,8378.0,5714.0,DELIVERED
4,2174675,68414018,10463.0,3746.0,DELIVERED
5,2174680,68415103,16430.0,3924.0,DELIVERED
6,2174693,68416643,14513.0,2489.0,DELIVERED
7,2174695,68417783,NaN,2564.0,CANCELLED
8,2174696,68415457,9996.0,340.0,DELIVERED
9,2174697,68414563,23092.0,1081.0,DELIVERED


In [77]:
data_deliveries.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378843 entries, 0 to 378842
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   delivery_id               378843 non-null  int64  
 1   delivery_order_id         378843 non-null  int64  
 2   driver_id                 362957 non-null  float64
 3   delivery_distance_meters  378770 non-null  float64
 4   delivery_status           378843 non-null  object 
dtypes: float64(2), int64(2), object(1)
memory usage: 14.5+ MB


* Kiểm tra giá trị null

In [78]:
data_deliveries.isnull().sum()


delivery_id                     0
delivery_order_id               0
driver_id                   15886
delivery_distance_meters       73
delivery_status                 0
dtype: int64

Tính phần trăm tỉ lệ dữ liệu thiếu

In [79]:
missing_data_per_deliveries = (data_deliveries.isnull().sum() / len(data_deliveries)) * 100
print("------ Tỉ lệ dữ liệu thiếu ------")
print(missing_data_per_deliveries.sort_values(ascending=False))

------ Tỉ lệ dữ liệu thiếu ------
driver_id                   4.193294
delivery_distance_meters    0.019269
delivery_id                 0.000000
delivery_order_id           0.000000
delivery_status             0.000000
dtype: float64


In [80]:
data_deliveries[data_deliveries['delivery_status'] == "CANCELLED"].head(10)

,delivery_id,delivery_order_id,driver_id,delivery_distance_meters,delivery_status
7,2174695,68417783,NaN,2564.0,CANCELLED
63,2174845,68424547,NaN,2100.0,CANCELLED
79,2174883,68427057,NaN,2995.0,CANCELLED
149,2175046,68435678,NaN,3958.0,CANCELLED
306,2175468,68437922,NaN,5460.0,CANCELLED
314,2175489,68457422,NaN,530.0,CANCELLED
342,2175562,68460384,NaN,5574.0,CANCELLED
359,2175600,68460544,NaN,2769.0,CANCELLED
374,2175643,68463805,NaN,694.0,CANCELLED
436,2175776,68445199,NaN,4234.0,CANCELLED


- Vì trong bảng `deliveries` có tồn tại khóa ngoại `driver_id` chứa giá trị NULL, khi kết nối với bảng Driver sẽ xảy ra lỗi. Do đó ta sẽ điền giá trị placeholder ( như -1) và chèn thêm 1 bảng ghi mới với `driver_id =-1`
- Đối với `delivery_distance_meters` thì điền giá trị trung vị

In [81]:
data_deliveries['driver_id'].fillna(-1, inplace=True)
data_deliveries['delivery_distance_meters'] = data_deliveries['delivery_distance_meters'].fillna(data_deliveries['delivery_distance_meters'].median()).astype('int64')

C:\Users\PC\AppData\Local\Temp\ipykernel_11772\1151317886.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_deliveries['driver_id'].fillna(-1, inplace=True)


In [82]:
data_deliveries['driver_id'] = data_deliveries['driver_id'].astype(int)

In [83]:
dataframes['deliveries'] = data_deliveries

In [84]:
data_deliveries.head(10)

,delivery_id,delivery_order_id,driver_id,delivery_distance_meters,delivery_status
0,2174658,68413340,8378,5199,DELIVERED
1,2174660,68414309,2473,410,DELIVERED
2,2174661,68416230,7615,3784,DELIVERED
3,2174663,68412721,8378,5714,DELIVERED
4,2174675,68414018,10463,3746,DELIVERED
5,2174680,68415103,16430,3924,DELIVERED
6,2174693,68416643,14513,2489,DELIVERED
7,2174695,68417783,-1,2564,CANCELLED
8,2174696,68415457,9996,340,DELIVERED
9,2174697,68414563,23092,1081,DELIVERED


In [85]:
dataframes['deliveries'].head(10)

,delivery_id,delivery_order_id,driver_id,delivery_distance_meters,delivery_status
0,2174658,68413340,8378,5199,DELIVERED
1,2174660,68414309,2473,410,DELIVERED
2,2174661,68416230,7615,3784,DELIVERED
3,2174663,68412721,8378,5714,DELIVERED
4,2174675,68414018,10463,3746,DELIVERED
5,2174680,68415103,16430,3924,DELIVERED
6,2174693,68416643,14513,2489,DELIVERED
7,2174695,68417783,-1,2564,CANCELLED
8,2174696,68415457,9996,340,DELIVERED
9,2174697,68414563,23092,1081,DELIVERED


In [86]:
data_deliveries.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378843 entries, 0 to 378842
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   delivery_id               378843 non-null  int64 
 1   delivery_order_id         378843 non-null  int64 
 2   driver_id                 378843 non-null  int64 
 3   delivery_distance_meters  378843 non-null  int64 
 4   delivery_status           378843 non-null  object
dtypes: int64(4), object(1)
memory usage: 14.5+ MB


In [87]:
data_deliveries.isnull().sum()  

delivery_id                 0
delivery_order_id           0
driver_id                   0
delivery_distance_meters    0
delivery_status             0
dtype: int64

## Xử lý bảng "Driver"

In [88]:
data_drivers.head(10)

,driver_id,driver_modal,driver_type
0,133,MOTOBOY,LOGISTIC OPERATOR
1,138,MOTOBOY,FREELANCE
2,140,MOTOBOY,FREELANCE
3,143,BIKER,FREELANCE
4,148,MOTOBOY,FREELANCE
5,165,MOTOBOY,FREELANCE
6,172,MOTOBOY,FREELANCE
7,174,BIKER,FREELANCE
8,187,BIKER,FREELANCE
9,196,BIKER,FREELANCE


In [89]:
data_drivers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4824 entries, 0 to 4823
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   driver_id     4824 non-null   int64 
 1   driver_modal  4824 non-null   object
 2   driver_type   4824 non-null   object
dtypes: int64(1), object(2)
memory usage: 113.2+ KB


* Kiểm tra null

In [90]:
data_drivers.isnull().sum()

driver_id       0
driver_modal    0
driver_type     0
dtype: int64

* Thêm một bảng ghi với `driver_id = -1`, `driver_modal = 'Unknown'`, `driver_type = 'Unknown'`, 

In [91]:
# 1. Định nghĩa bản ghi 'Unknown' mới
new_unknown_driver = {
    'driver_id': -1,  # Sử dụng -1 làm ID cho tài xế không xác định
    'driver_modal': 'N/A', 
    'driver_type': 'N/A',
    # Thêm các cột khác của bảng drivers và điền giá trị 'N/A' hoặc 'Unknown'
}

# Chuyển thành DataFrame (cần đảm bảo các tên cột khớp với data_drivers)
new_driver_df = pd.DataFrame([new_unknown_driver])

# 2. Thêm bản ghi này vào bảng drivers gốc
# Sử dụng pd.concat() để thêm hàng mới (tài xế -1) vào data_drivers
data_drivers = pd.concat([data_drivers, new_driver_df], ignore_index=True)
data_drivers.drop_duplicates(subset=['driver_id'], keep='first', inplace=True)
# Kiểm tra xem bản ghi -1 đã tồn tại chưa
print("\nKiểm tra bản ghi -1 trong data_drivers:")
print(data_drivers[data_drivers['driver_id'] == -1])


Kiểm tra bản ghi -1 trong data_drivers:
      driver_id driver_modal driver_type
4824         -1          N/A         N/A


In [92]:
data_drivers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4825 entries, 0 to 4824
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   driver_id     4825 non-null   int64 
 1   driver_modal  4825 non-null   object
 2   driver_type   4825 non-null   object
dtypes: int64(1), object(2)
memory usage: 113.2+ KB


In [93]:
data_drivers[data_drivers['driver_id'] == -1].head(10)

,driver_id,driver_modal,driver_type
4824,-1,N/A,N/A


In [94]:
dataframes['drivers'] = data_drivers

## Xử lý bảng "Hubs

In [95]:
dataframes['hubs'].head(10)

,hub_id,hub_name,hub_city,hub_state,hub_latitude,hub_longitude
0,2,BLUE SHOPPING,PORTO ALEGRE,RS,-30.047415,-51.213510
1,3,GREEN SHOPPING,PORTO ALEGRE,RS,-30.037415,-51.203520
2,4,RED SHOPPING,PORTO ALEGRE,RS,-30.021948,-51.208382
3,5,FUNK SHOPPING,RIO DE JANEIRO,RJ,-23.000750,-43.318282
4,8,GOLDEN SHOPPING,RIO DE JANEIRO,RJ,-22.921475,-43.234774
5,13,HIP HOP SHOPPING,RIO DE JANEIRO,RJ,-22.885820,-43.279218
6,16,PEOPLE SHOPPING,RIO DE JANEIRO,RJ,-23.017472,-43.479939
7,17,SMALL SHOPPING,SÃO PAULO,SP,-23.592004,-46.636504
8,18,STAR SHOPPING,RIO DE JANEIRO,RJ,-22.945495,-43.182181
9,20,PURPLE SHOPPING,RIO DE JANEIRO,RJ,-22.996848,-43.359636


In [96]:
dataframes['hubs'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   hub_id         32 non-null     int64  
 1   hub_name       32 non-null     object 
 2   hub_city       32 non-null     object 
 3   hub_state      32 non-null     object 
 4   hub_latitude   32 non-null     float64
 5   hub_longitude  32 non-null     float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.6+ KB


## Xử lý bảng Stores

In [97]:
data_stores = dataframes['stores']
data_stores.head(20)

,store_id,hub_id,store_name,store_segment,store_plan_price,store_latitude,store_longitude
0,3,2,CUMIURI,FOOD,0.0,NaN,NaN
1,6,3,PIMGUCIS DA VIVA,FOOD,0.0,-30.037415,-51.203520
2,8,3,RASMUR S,FOOD,0.0,-30.037415,-51.203520
3,53,8,PAPA SUCIS,FOOD,0.0,-22.921475,-43.234822
4,54,8,VUZPI PAZZIS,FOOD,0.0,-22.921475,-43.234822
5,56,8,SUPSIO,FOOD,49.0,-22.921475,-43.234822
6,58,8,PIAMUARIS,FOOD,49.0,-22.921475,-43.234822
7,82,8,LUCITA,FOOD,0.0,-22.921475,-43.234822
8,83,8,PRARIZZAI,FOOD,0.0,-22.921475,-43.234822
9,84,8,PALLO MZU GRALA,FOOD,49.0,-22.921475,-43.234822


In [98]:
data_stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 951 entries, 0 to 950
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   store_id          951 non-null    int64  
 1   hub_id            951 non-null    int64  
 2   store_name        951 non-null    object 
 3   store_segment     951 non-null    object 
 4   store_plan_price  836 non-null    float64
 5   store_latitude    935 non-null    float64
 6   store_longitude   935 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 52.1+ KB


In [99]:
data_stores.isnull().sum()

store_id              0
hub_id                0
store_name            0
store_segment         0
store_plan_price    115
store_latitude       16
store_longitude      16
dtype: int64

In [100]:
data_stores[['store_id','store_plan_price','hub_id']][data_stores["store_plan_price"].isnull()].head(5)

,store_id,store_plan_price,hub_id
59,415,NaN,8
62,426,NaN,20
76,496,NaN,26
95,587,NaN,8
99,599,NaN,21


* Xử lý NULL
  * Ở đây tọa độ của một số của hàng bị NULL, thay vì xóa đi thì ta sẽ join 2 bảng là `hubs` và `stores` lại với nhau rồi điền NULL bằng tọa độ của `hubs`
  * Những cửa hàng chưa có giá dự bán thì sẽ để bằng 0

In [101]:
data_hubs = dataframes['hubs']
data_stores = dataframes['stores']

df_hubs_coords = data_hubs[['hub_id', 'hub_latitude', 'hub_longitude']]


df_hubs_coords = df_hubs_coords.rename(columns={
    'hub_latitude': 'matched_latitude',
    'hub_longitude': 'matched_longitude'
})

# Thực hiện Left Join (kết hợp trái)
# Điều này giữ lại TẤT CẢ các hàng từ df_stores và thêm tọa độ hub tương ứng.
df_merged = data_stores.merge(
    df_hubs_coords,
    on='hub_id',
    how='left'
)
df_merged.head(20)

,store_id,hub_id,store_name,store_segment,store_plan_price,store_latitude,store_longitude,matched_latitude,matched_longitude
0,3,2,CUMIURI,FOOD,0.0,NaN,NaN,-30.047415,-51.213510
1,6,3,PIMGUCIS DA VIVA,FOOD,0.0,-30.037415,-51.203520,-30.037415,-51.203520
2,8,3,RASMUR S,FOOD,0.0,-30.037415,-51.203520,-30.037415,-51.203520
3,53,8,PAPA SUCIS,FOOD,0.0,-22.921475,-43.234822,-22.921475,-43.234774
4,54,8,VUZPI PAZZIS,FOOD,0.0,-22.921475,-43.234822,-22.921475,-43.234774
5,56,8,SUPSIO,FOOD,49.0,-22.921475,-43.234822,-22.921475,-43.234774
6,58,8,PIAMUARIS,FOOD,49.0,-22.921475,-43.234822,-22.921475,-43.234774
7,82,8,LUCITA,FOOD,0.0,-22.921475,-43.234822,-22.921475,-43.234774
8,83,8,PRARIZZAI,FOOD,0.0,-22.921475,-43.234822,-22.921475,-43.234774
9,84,8,PALLO MZU GRALA,FOOD,49.0,-22.921475,-43.234822,-22.921475,-43.234774


In [102]:

df_merged['store_latitude'] = df_merged['store_latitude'].fillna(
    df_merged['matched_latitude']
)
df_merged['store_longitude'] = df_merged['store_longitude'].fillna(
    df_merged['matched_longitude']
)
df_merged['store_plan_price'] = df_merged['store_plan_price'].fillna(0)
df_merged.drop(columns=['matched_latitude', 'matched_longitude'], inplace=True)
df_merged.head(20)

,store_id,hub_id,store_name,store_segment,store_plan_price,store_latitude,store_longitude
0,3,2,CUMIURI,FOOD,0.0,-30.047415,-51.213510
1,6,3,PIMGUCIS DA VIVA,FOOD,0.0,-30.037415,-51.203520
2,8,3,RASMUR S,FOOD,0.0,-30.037415,-51.203520
3,53,8,PAPA SUCIS,FOOD,0.0,-22.921475,-43.234822
4,54,8,VUZPI PAZZIS,FOOD,0.0,-22.921475,-43.234822
5,56,8,SUPSIO,FOOD,49.0,-22.921475,-43.234822
6,58,8,PIAMUARIS,FOOD,49.0,-22.921475,-43.234822
7,82,8,LUCITA,FOOD,0.0,-22.921475,-43.234822
8,83,8,PRARIZZAI,FOOD,0.0,-22.921475,-43.234822
9,84,8,PALLO MZU GRALA,FOOD,49.0,-22.921475,-43.234822


In [103]:
df_merged[['store_id','store_plan_price','hub_id']][df_merged["store_id"].isin([415,426,496,587,599])].head(5)

,store_id,store_plan_price,hub_id
59,415,0.0,8
62,426,0.0,20
76,496,0.0,26
95,587,0.0,8
99,599,0.0,21


In [104]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 951 entries, 0 to 950
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   store_id          951 non-null    int64  
 1   hub_id            951 non-null    int64  
 2   store_name        951 non-null    object 
 3   store_segment     951 non-null    object 
 4   store_plan_price  951 non-null    float64
 5   store_latitude    951 non-null    float64
 6   store_longitude   951 non-null    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 52.1+ KB


In [105]:
dataframes['stores'] = df_merged

## Xử lý Payments

In [106]:
data_payments = dataframes['payments']

In [107]:
data_payments.head(10)

,payment_id,payment_order_id,payment_amount,payment_fee,payment_method,payment_status
0,4427917,68410055,118.44,0.00,VOUCHER,PAID
1,4427918,68410055,394.81,7.90,ONLINE,PAID
2,4427941,68412721,206.95,5.59,ONLINE,PAID
3,4427948,68413340,58.80,1.59,ONLINE,PAID
4,4427955,68414018,45.80,0.92,ONLINE,PAID
5,4427956,68414309,106.80,2.88,ONLINE,PAID
6,4427961,68414512,57.80,1.56,ONLINE,PAID
7,4427963,68414563,26.90,0.40,ONLINE,PAID
8,4427975,68415103,115.50,3.12,ONLINE,PAID
9,4427976,68415140,199.30,5.38,ONLINE,PAID


In [108]:
data_payments[data_payments["payment_method"].isnull()].head(10)

,payment_id,payment_order_id,payment_amount,payment_fee,payment_method,payment_status


In [109]:
data_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400834 entries, 0 to 400833
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   payment_id        400834 non-null  int64  
 1   payment_order_id  400834 non-null  int64  
 2   payment_amount    400834 non-null  float64
 3   payment_fee       400659 non-null  float64
 4   payment_method    400834 non-null  object 
 5   payment_status    400834 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 18.3+ MB


* Kiểm tra NULL

In [110]:
data_payments.isnull().sum()

payment_id            0
payment_order_id      0
payment_amount        0
payment_fee         175
payment_method        0
payment_status        0
dtype: int64

In [111]:
data_payments[['payment_id','payment_amount','payment_fee','payment_method','payment_status']][data_payments['payment_fee'].isnull() & (data_payments['payment_method'] != "ONLINE" ) ]

,payment_id,payment_amount,payment_fee,payment_method,payment_status
124607,4753892,185.1,NaN,CREDIT_STORE,PAID
189726,4926706,159.8,NaN,STORE_DIRECT_PAYMENT,PAID
231658,5036577,31.9,NaN,CREDIT,PAID
232098,5037651,184.6,NaN,DEBIT,PAID
245393,5070972,309.9,NaN,VOUCHER_DC,PAID
245394,5070975,309.9,NaN,VOUCHER_DC,PAID
246815,5074765,9.9,NaN,DEBIT,PAID
247418,5076402,114.8,NaN,DEBIT,PAID
248170,5078195,9.9,NaN,CREDIT,PAID
248967,5080212,199.6,NaN,CREDIT,PAID


* Sử dụng hồi quy tuyến tính để điền các giá trị NULL

In [112]:

data_train = data_payments[data_payments['payment_fee'].notna()].copy()
data_predict = data_payments[data_payments['payment_fee'].isna()].copy()
# Xác định các biến độc lập (features)
FEATURES = ['payment_amount', 'payment_method','payment_status']
TARGET = 'payment_fee'
X_train = data_train[FEATURES]
y_train = data_train[TARGET]
# One-Hot Encoding cho payment_method và payment_status
X_train_encoded = pd.get_dummies(X_train, columns=['payment_method','payment_status'], drop_first=True)
# Lấy danh sách cột sau khi mã hóa để đảm bảo khớp với tập dự đoán
encoded_cols = X_train_encoded.columns
# Khởi tạo mô hình Hồi quy Tuyến tính
model = LinearRegression()
# Huấn luyện mô hình
model.fit(X_train_encoded, y_train)
print(f"\nĐã huấn luyện mô hình Hồi quy Tuyến tính với R^2 là: {model.score(X_train_encoded, y_train):.4f}")


Đã huấn luyện mô hình Hồi quy Tuyến tính với R^2 là: 0.9695


In [113]:
# 1. Lấy features từ tập dữ liệu cần dự đoán
X_predict = data_predict[FEATURES]
X_predict_encoded = pd.get_dummies(X_predict, columns=['payment_method'], drop_first=True)
X_predict_final = pd.DataFrame(0, index=X_predict_encoded.index, columns=encoded_cols)
for col in X_predict_encoded.columns:
    if col in X_predict_final.columns:
        X_predict_final[col] = X_predict_encoded[col]

In [114]:
# 1. Dự đoán payment_fee
predicted_fees = model.predict(X_predict_final)
predicted_fees[predicted_fees < 0] = 0.00 # Đảm bảo phí không âm (vì đây là chi phí)

# 2. Điền giá trị đã dự đoán trở lại data_payments gốc
data_payments.loc[data_predict.index, 'payment_fee'] = predicted_fees

In [115]:
data_payments['payment_fee'] = data_payments['payment_fee'].round(2)
data_payments[['payment_id','payment_amount','payment_fee','payment_method','payment_status']][data_payments['payment_id'].isin([4753892, 4926706, 5036577, 5037651, 5070972])]

,payment_id,payment_amount,payment_fee,payment_method,payment_status
124607,4753892,185.1,3.32,CREDIT_STORE,PAID
189726,4926706,159.8,3.07,STORE_DIRECT_PAYMENT,PAID
231658,5036577,31.9,0.27,CREDIT,PAID
232098,5037651,184.6,3.40,DEBIT,PAID
245393,5070972,309.9,5.29,VOUCHER_DC,PAID


In [116]:
data_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400834 entries, 0 to 400833
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   payment_id        400834 non-null  int64  
 1   payment_order_id  400834 non-null  int64  
 2   payment_amount    400834 non-null  float64
 3   payment_fee       400834 non-null  float64
 4   payment_method    400834 non-null  object 
 5   payment_status    400834 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 18.3+ MB


In [ ]:
dataframes['payments'] = data_payments

---------

# Kiểm tra bộ dữ liệu lần cuối

In [31]:
data = ['orders', 'deliveries', 'drivers', 'hubs', 'stores', 'payments', 'channels']
for d in data:
    print(f"\n--- Thông tin bảng '{d}' ---")
    dataframes[d].info()


--- Thông tin bảng 'orders' ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368999 entries, 0 to 368998
Data columns (total 30 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   order_id                           368999 non-null  int64         
 1   store_id                           368999 non-null  int64         
 2   channel_id                         368999 non-null  int64         
 3   order_status                       368999 non-null  object        
 4   order_amount                       368999 non-null  float64       
 5   order_delivery_fee                 368999 non-null  float64       
 6   order_delivery_cost                368999 non-null  float64       
 7   order_created_hour                 368999 non-null  int64         
 8   order_created_minute               368999 non-null  int64         
 9   order_created_day                  368999 non-null  int64  

# Xuất ra file csv

In [30]:

def export_all_dfs_to_csv(
    dataframes: Dict[str, pd.DataFrame],
    output_dir: str = 'exported_csv',
    index: bool = False
) -> None:
    """
    Xuất tất cả các Pandas DataFrame trong dictionary ra các tệp CSV.

    Args:
        dataframes (Dict[str, pd.DataFrame]): Dictionary chứa tên (key) 
                                              và DataFrame (value) cần xuất.
        output_dir (str): Tên thư mục để lưu các tệp CSV. Thư mục sẽ được tạo
                          nếu chưa tồn tại. Mặc định là 'exported_csv'.
        index (bool): Liệu có bao gồm index (chỉ mục) của DataFrame trong tệp CSV hay không.
                      Mặc định là False.
    """
    
    # 1. Tạo thư mục đầu ra nếu chưa tồn tại
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"📁 Đã tạo thư mục: {output_dir}")

    # 2. Lặp qua từng DataFrame và lưu
    for df_name, df in dataframes.items():
        # Tạo tên tệp CSV: ví dụ 'stores.csv'
        filename = f"{df_name}.csv"
        filepath = os.path.join(output_dir, filename)
        
        try:
            # Sử dụng phương thức to_csv của Pandas để lưu
            df.to_csv(filepath, index=index)
            print(f"✅ Đã lưu '{df_name}' vào: {filepath}")
            
        except Exception as e:
            print(f"❌ Lỗi khi lưu DataFrame '{df_name}': {e}")
    
    print("\n🎉 Hoàn tất quá trình xuất CSV.")


In [33]:
all_dataframes = {
#     #"stores": dataframes['stores'],
#     #"hubs": dataframes['hubs'],
    "orders": dataframes['orders'] 
#     #"deliveries": dataframes['deliveries'],
#     #"drivers": dataframes['drivers'],
#     #"payments": dataframes['payments'],
#     #"channels": dataframes['channels']
}

export_all_dfs_to_csv(all_dataframes, output_dir='etl_output')

✅ Đã lưu 'orders' vào: etl_output\orders.csv

🎉 Hoàn tất quá trình xuất CSV.


# Đẩy lên MySQL

In [121]:

DB_HOST = '127.0.0.1' 
DB_USER = 'root' 
DB_PASSWORD = 'quangduc101112'
DB_NAME = 'ETL_python' 
# Xây dựng chuỗi kết nối (Connection String)
connection_string = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"
# Tạo Engine (công cụ kết nối)
try:
    engine = create_engine(connection_string)
    print("Kết nối MySQL Engine đã được tạo thành công.")
except Exception as e:
    print(f"Lỗi tạo Engine: {e}")

Kết nối MySQL Engine đã được tạo thành công.


In [ ]:

# # Thông tin kết nối MySQL của bạn
# DB_HOST = '127.0.0.1'  # Ví dụ: 'localhost' hoặc địa chỉ IP
# DB_USER = 'root'  # Ví dụ: 'root'
# DB_PASSWORD = 'quangduc101112'
# DB_NAME = 'raw' # Ví dụ: 'etl_project'

# # Xây dựng chuỗi kết nối (Connection String)
# # Sử dụng 'mysql+pymysql'
# connection_string = f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"

# # Tạo Engine (công cụ kết nối)
# try:
#     engine = create_engine(connection_string)
#     print("✅ Kết nối MySQL Engine đã được tạo thành công.")
# except Exception as e:
#     print(f"❌ Lỗi tạo Engine: {e}")

✅ Kết nối MySQL Engine đã được tạo thành công.


In [123]:

for d in data:
    try:
        TABLE_NAME = d
        dataframes[d].to_sql(
            name=TABLE_NAME,
            con=engine,
            if_exists='replace',  # Rất quan trọng:
        index=False
    )
        print(f"Tải thành công lên bảng '{TABLE_NAME}'.")
    
    except Exception as e:
        print(f"Lỗi khi tải dữ liệu lên MySQL: {e}")

    finally:
        # Đóng kết nối (rất quan trọng)
        engine.dispose()
        print("Đã đóng kết nối database.")

Tải thành công lên bảng 'orders'.
Đã đóng kết nối database.
Tải thành công lên bảng 'deliveries'.
Đã đóng kết nối database.
Tải thành công lên bảng 'drivers'.
Đã đóng kết nối database.
Tải thành công lên bảng 'hubs'.
Đã đóng kết nối database.
Tải thành công lên bảng 'stores'.
Đã đóng kết nối database.
Tải thành công lên bảng 'payments'.
Đã đóng kết nối database.
Tải thành công lên bảng 'channels'.
Đã đóng kết nối database.
